In [1]:
#!/usr/bin/env python
__author__="Vitali Molchan"

import sys

from bs4 import BeautifulSoup
from string import find,strip, translate

import glob
import time
import dateutil

import re
import pandas as pd
import numpy as np

import matplotlib.pylab as plt
import seaborn as sns; sns.set()
import difflib

In [2]:
cbsBuurt2015 = pd.read_pickle("Processed_data/cbsBuurt2015.pickle")
funda = pd.read_pickle("Processed_data/funda.pickle")

In [4]:
zipcodes = pd.read_csv("ZIPCODES/bagadres.csv",sep=";",header=0,\
                       names=['OPENBARERUIMTE','HOUSE_NUMBER','HOUSE_LETTER','HOUSE_SUFFIX','POSTCODE','WOONPLATS','GEMEENTE','PROVINCE','OBJECT_ID','OBJECT_TYPE','SECONDARY_ADDRESS','X','Y','LON','LAT'],\
                       dtype={'OPENBARERUIMTE':str,'HOUSE_NUMBER':np.int,'HOUSE_LETTER':str,'HOUSE_SUFFIX':str,'POSTCODE':str,'WOONPLATS':str,'GEMEENTE':str,'PROVINCE':str,'OBJECT_ID':np.int64,'OBJECT_TYPE':str,'SECONDARY_ADDRESS':str,'X':np.float,'Y':np.float,'LON':np.float,'LAT':np.float})

In [5]:
cbsBuurt2015.head()

,GM_NAAM,BU_NAAM,POSTCODE,BU_CODE,WK_CODE,GM_CODE,WATER,AANTAL_HH,AANT_INW,AANT_MAN,...,P_VERWEDUW,P_WEST_AL,P_WONT2000,P_WONV2000,STED,STERFT_TOT,WONINGEN,WOZ,WWB_UITTOT,WW_UIT_TOT
0,'s-Gravenhage,Archipelbuurt,2585,BU05180546,WK051805,GM0518,NEE,3210.0,5720.0,2700.0,...,4.0,30.0,94.0,6.0,1.0,35.0,3591.0,376.0,60.0,90.0
1,'s-Gravenhage,Arendsdorp,2596,BU05180478,WK051804,GM0518,NEE,555.0,1310.0,625.0,...,9.0,29.0,100.0,0.0,1.0,35.0,866.0,483.0,10.0,20.0
2,'s-Gravenhage,Belgisch Park,2587,BU05180271,WK051802,GM0518,NEE,4130.0,7955.0,3935.0,...,5.0,23.0,99.0,1.0,2.0,110.0,4091.0,294.0,120.0,130.0
3,'s-Gravenhage,Bezuidenhout-Midden,2595,BU05182665,WK051826,GM0518,NEE,2120.0,3970.0,2000.0,...,3.0,27.0,80.0,20.0,1.0,20.0,2069.0,244.0,30.0,80.0
4,'s-Gravenhage,Bezuidenhout-Oost,2593,BU05182666,WK051826,GM0518,NEE,5175.0,8940.0,4315.0,...,3.0,22.0,98.0,2.0,1.0,45.0,5151.0,169.0,210.0,200.0


In [6]:
funda.head()

,FILE_NAME,PROPERTY_TYPE,PROPERTY_ID,POSTCODE,GEMEENTE,BUURT,OFFER_DATE,SOLD_DATE,SOLD_PRICE,ASKING_PRICE,...,GARDEN_TYPE,GARDEN_AREA,PARKING_TYPE,ENERGY_LABEL,MAKLER_NAME,MAKLER_TEL,HOUSE_LETTER,HOUSE_NUMBER,STREET,GARDEN_FLAG
0,appartement-4003351-boerhaavelaan-102-a-featur...,appartement,4003351,3112LM,Schiedam,Rotterdamsedijk,2008-01-15,2015-08-08,129000.0,149000.0,...,Achtertuin,50.0,Openbaar parkeren en betaald parkeren,C,Makelaardij Wassenaar,010-4737933,A,102.0,BOERHAAVELAAN,1.0
1,appartement-4043458-a-weg-10-25-features.html,appartement,4043458,9581AK,Musselkanaal,Zuid,2007-11-06,2016-03-14,247500.0,286500.0,...,NaN,NaN,NaN,B,Kral Westen Makelaardij B.V.,0599-581759,NaN,10.0,A WEG,0.0
2,appartement-4047711-dingstraat-20-features.html,appartement,4047711,7101GX,Winterswijk,Centrale deel,2007-10-31,2015-11-11,180000.0,265000.0,...,NaN,NaN,NaN,B,Enning Makelaardij b.v.,0543-516151,NaN,20.0,DINGSTRAAT,0.0
3,appartement-4096883-groningerstraat-72-d-featu...,appartement,4096883,9401BM,Assen,Galgenveld,2007-12-19,2016-01-04,175000.0,267550.0,...,NaN,NaN,NaN,A,Oosting Makelaars,0592-310144,D,72.0,GRONINGERSTRAAT,0.0
4,appartement-4104014-weverstraat-11-features.html,appartement,4104014,7271AJ,Borculo,Borculo Oude-Centrum,2008-04-22,2015-09-21,125000.0,158500.0,...,NaN,NaN,NaN,C,Bijker makelaardij,0545-254444,NaN,11.0,WEVERSTRAAT,0.0


In [7]:
zipcodes.head()

,OPENBARERUIMTE,HOUSE_NUMBER,HOUSE_LETTER,HOUSE_SUFFIX,POSTCODE,WOONPLATS,GEMEENTE,PROVINCE,OBJECT_ID,OBJECT_TYPE,SECONDARY_ADDRESS,X,Y,LON,LAT
0,De Ruijterkade,105,NaN,1,1011AB,Amsterdam,Amsterdam,Noord-Holland,363010000964973,VBO,f,122177.0,487877.0,4.905167,52.377780
1,De Ruijterkade,105,NaN,2,1011AB,Amsterdam,Amsterdam,Noord-Holland,363010000784610,VBO,f,122177.0,487877.0,4.905167,52.377780
2,De Ruijterkade,105,NaN,3,1011AB,Amsterdam,Amsterdam,Noord-Holland,363010000977861,VBO,f,122177.0,487877.0,4.905167,52.377780
3,De Ruijterkade,105,NaN,H,1011AB,Amsterdam,Amsterdam,Noord-Holland,363010000785105,VBO,f,122177.0,487877.0,4.905167,52.377780
4,De Ruijterkade,106,NaN,1,1011AB,Amsterdam,Amsterdam,Noord-Holland,363010012064067,VBO,f,122185.0,487882.9,4.905284,52.377833


In [8]:
print "Shape of CBS file with Buurt info is {}".format(cbsBuurt2015.shape)
print "Shape of Funda database  is {}".format(funda.shape)
print "Shape of PostCode database  is {}".format(zipcodes.shape)

Shape of CBS file with Buurt info is (12179, 127)
Shape of Funda database  is (195288, 30)
Shape of PostCode database  is (8888072, 15)


In [9]:
# Deleting rows with undefined SOLD_PRICE attribute
funda = funda.dropna(subset=['SOLD_PRICE'])

# Check the shape
print "Shape of Funda database  is {}".format(funda.shape)

Shape of Funda database  is (195288, 30)


Around 12000 empty rows were deleted

Count the number of items of different types (str, digit, na) in each column

In [ ]:
df.applymap(lambda x: isinstance(x, str)).sum()

In [ ]:
df.applymap(lambda x: isinstance(x, (str))).sum()

Combine two dataframe based on POSTCODE attribute (POSTCODE in cbsBuurt2015 and POSTCODE_NUMBERS in funda).

In [13]:
def extract_digit_from_postcode(x):
    if pd.isnull(x)==False and x.isalnum():
        try:
            return int(x[0:4])
        except:
            return 0
    else:
        return 0
    
funda['POSTCODE_NUMBERS'] = funda['POSTCODE'].apply(extract_digit_from_postcode)

In [14]:
funda.head()

,FILE_NAME,PROPERTY_TYPE,PROPERTY_ID,POSTCODE,GEMEENTE,BUURT,OFFER_DATE,SOLD_DATE,SOLD_PRICE,ASKING_PRICE,...,GARDEN_AREA,PARKING_TYPE,ENERGY_LABEL,MAKLER_NAME,MAKLER_TEL,HOUSE_LETTER,HOUSE_NUMBER,STREET,GARDEN_FLAG,POSTCODE_NUMBERS
0,appartement-4003351-boerhaavelaan-102-a-featur...,appartement,4003351,3112LM,Schiedam,Rotterdamsedijk,2008-01-15,2015-08-08,129000.0,149000.0,...,50.0,Openbaar parkeren en betaald parkeren,C,Makelaardij Wassenaar,010-4737933,A,102.0,BOERHAAVELAAN,1.0,3112
1,appartement-4043458-a-weg-10-25-features.html,appartement,4043458,9581AK,Musselkanaal,Zuid,2007-11-06,2016-03-14,247500.0,286500.0,...,NaN,NaN,B,Kral Westen Makelaardij B.V.,0599-581759,NaN,10.0,A WEG,0.0,9581
2,appartement-4047711-dingstraat-20-features.html,appartement,4047711,7101GX,Winterswijk,Centrale deel,2007-10-31,2015-11-11,180000.0,265000.0,...,NaN,NaN,B,Enning Makelaardij b.v.,0543-516151,NaN,20.0,DINGSTRAAT,0.0,7101
3,appartement-4096883-groningerstraat-72-d-featu...,appartement,4096883,9401BM,Assen,Galgenveld,2007-12-19,2016-01-04,175000.0,267550.0,...,NaN,NaN,A,Oosting Makelaars,0592-310144,D,72.0,GRONINGERSTRAAT,0.0,9401
4,appartement-4104014-weverstraat-11-features.html,appartement,4104014,7271AJ,Borculo,Borculo Oude-Centrum,2008-04-22,2015-09-21,125000.0,158500.0,...,NaN,NaN,C,Bijker makelaardij,0545-254444,NaN,11.0,WEVERSTRAAT,0.0,7271


In [15]:
#First, let's check does CBS database contains all necessary postcodes as in funda database
missed_postcodes = set(funda['POSTCODE_NUMBERS'].unique())-set(cbsBuurt2015['POSTCODE'].unique())
print "Number of missed postcodes is {}".format(len(missed_postcodes))
print sorted(missed_postcodes)

Number of missed postcodes is 111
[0, 1022, 1026, 1027, 1028, 1032, 1036, 1059, 1074, 1081, 1086, 1104, 1107, 1300, 1352, 1428, 1609, 1642, 1643, 1663, 1683, 1684, 1847, 1989, 2019, 2172, 2546, 2561, 2632, 2693, 2726, 2872, 2981, 3023, 3036, 3052, 3069, 3078, 3191, 3193, 3340, 3373, 3642, 3643, 3800, 3900, 3962, 4017, 4103, 4104, 4106, 4107, 4163, 4212, 4302, 4482, 4522, 4536, 4537, 4652, 4722, 4772, 4781, 5050, 5057, 5063, 5122, 5252, 5293, 5318, 5381, 5392, 5449, 5689, 6092, 6111, 6370, 6469, 6534, 6575, 6577, 7048, 7105, 7113, 7119, 7392, 7472, 7525, 7702, 7859, 7937, 7966, 7985, 8082, 8254, 8337, 8339, 8444, 8452, 8472, 8532, 8702, 8893, 9085, 9302, 9412, 9413, 9623, 9624, 9841, 9882]


There are 110 postcodes (without counting 0 value) are not presented in CBS dataframe. 

In [16]:
s=0
postcode_funda2cbs_converter = {}
error_postcodes = []
for postcode in missed_postcodes:
    
    s+=funda['POSTCODE_NUMBERS'][funda['POSTCODE_NUMBERS']==postcode].count()
    df = funda[funda['POSTCODE_NUMBERS']==postcode].groupby(['GEMEENTE','BUURT']).size().reset_index().rename(columns={0:'count'})
    
    for idx in range(len(df)):
        gemeente = df[['GEMEENTE']].iloc[idx,:].values[0]
        buurt = df[['BUURT']].iloc[idx,:].values[0]
        cbs_postcode = cbsBuurt2015['POSTCODE'][(cbsBuurt2015['GM_NAAM']==gemeente)&(cbsBuurt2015['BU_NAAM']==buurt)].values
        
        if cbs_postcode>0:
            postcode_funda2cbs_converter[postcode] = cbs_postcode
            print gemeente,buurt,postcode,cbs_postcode
        else:
            error_postcodes.append((gemeente,buurt))
            print gemeente,buurt,postcode,cbs_postcode
    
print "\nTotal number of sold properties with suspicious postcodes is {}".format(s)

Harde 't Harde-West 0 []
Den Bloemenbuurt-Oost 2561 []
Den Bomenbuurt 2561 []
Den Heesterbuurt 2561 []
Den Koningsplein en omgeving 2561 []
Den Valkenboskwartier 2561 []
Amsterdam Waterland 1026 [1023]
Amsterdam Waterland 1027 [1023]
Amsterdam Waterland 1028 [1023]
Rotterdam Groot-IJsselmonde 3078 []
Culemborg Oude Buitenwijken 4103 [4101]
Culemborg Verspreide huizen ten westen en zuiden van Culemborg 4103 [4105]
Amsterdam Buiksloterham 1032 [1033]
Amsterdam Volewijck 1032 [1031]
Culemborg Verspreide huizen ten westen en zuiden van Culemborg 4106 [4105]
Culemborg Verspreide huizen ten westen en zuiden van Culemborg 4107 [4105]
Amsterdam Tuindorp Oostzaan 1036 [1033]
Rijen Rijen 5122 []
Gemonde Gemonde 5293 []
Dedemsvaart Dedemsvaart-Zuidwest 7702 []
Amsterdam Hoofddorppleinbuurt 1059 [1058]
Amsterdam Westlandgracht 1059 [1062]
Erlecom Verspreide huizen Ooij, Persingen, Erlecom 6577 []
Steenbergen Steenbergen-Noord 4652 [4651]
Culemborg Industriegebied 4104 [4105]
Amsterdam Diamantbuurt

In [ ]:
# Many correct postcodes can be found in CBS database based on GEMEENTE and BUURT values but not all of them
# Replace postcode in funda with correct one from CBS
for funda_postcode,cbs_postcode in postcode_funda2cbs_converter.iteritems():
    funda['POSTCODE_NUMBERS'][funda['POSTCODE_NUMBERS']==funda_postcode]=cbs_postcode

In [18]:
# Make an attempt to find the gemeentes and buurts in CBS dataframe which is spell similarly to suspisious data in Funda
for items in error_postcodes:
    gemeente = items[0]
    buurt = items[1]
    
    all_gemeentes = cbsBuurt2015['GM_NAAM'].unique()
    #check if gemeente in cbs database and find the most similar gemeente using difflib library
    print gemeente,buurt,  gemeente in all_gemeentes, difflib.get_close_matches(gemeente,all_gemeentes,cutoff=0.5)
    
    # if gemeente in cbs database find buurs with "the closets" spelling
    if gemeente in all_gemeentes:
        all_gemeente_buurts = cbsBuurt2015[cbsBuurt2015['GM_NAAM']==gemeente]['BU_NAAM'].values
        print "List of similar buurts for {} is {}\n".format(buurt, difflib.get_close_matches(buurt,all_gemeente_buurts,cutoff=0.2))
        

Harde 't Harde-West False [u'Haren', u'Heerde', u'Haaren']
Den Bloemenbuurt-Oost False [u'Duiven', u'Druten', u'Dongen']
Den Bomenbuurt False [u'Duiven', u'Druten', u'Dongen']
Den Heesterbuurt False [u'Duiven', u'Druten', u'Dongen']
Den Koningsplein en omgeving False [u'Duiven', u'Druten', u'Dongen']
Den Valkenboskwartier False [u'Duiven', u'Druten', u'Dongen']
Rotterdam Groot-IJsselmonde True [u'Rotterdam', u'Roerdalen', u'Amsterdam']
List of similar buurts for Groot-IJsselmonde is [u'Groot IJsselmonde', u'Oud IJsselmonde', u'Nesselande']

Rijen Rijen False [u'Wijchen', u'Strijen', u'Rijnwaarden']
Gemonde Gemonde False [u'Helmond', u'Roermond', u'Eemsmond']
Dedemsvaart Dedemsvaart-Zuidwest False [u'Zevenaar', u'Beemster', u'Weert']
Erlecom Verspreide huizen Ooij, Persingen, Erlecom False [u'Haarlem', u'Hillegom']
Mijdrecht Mijdrecht False [u'Zwijndrecht', u'Sliedrecht', u'Dordrecht']
Mijdrecht Mijdrecht False [u'Zwijndrecht', u'Sliedrecht', u'Dordrecht']
Oosterwijk Kedichem False [u'O

Several possible matches can be found:
1. Funda (gemeente - Rotterdam, buurt - Groot-IJsselmonde) --> CBS (gemeente - Rotterdam, buurt - Groot IJsselmonde)
2. Funda (gemeente - Ridderkerk, buurt - Ridderkerk-Centrum) --> CBS (gemeente - Ridderkerk, buurt - Centrum)
3. Funda (gemeente - Ridderkerk, buurt - Ridderkerk-Oost) --> CBS (gemeente - Ridderkerk, buurt - Oost)
4. Funda (gemeente - Ridderkerk, buurt - Ridderkerk-West) --> CBS (gemeente - Ridderkerk, buurt - West)
5. Funda (gemeente - Oisterwijk, buurt - Pannenschuur ) --> CBS (gemeente - Oisterwijk. buurt - Omg.Pannenschuur-Centrum)
6. Funda (gemeente - Nootdorp) --> CBS (gemeente - Pijnacker-Nootdorp)
 
<br />Such type of task needs manual analysis and cannot be performed in automatic way easily.

In [ ]:
# Replace Funda gemeente/buurt data with correct CBS data
funda['BUURT'][(funda['GEMEENTE']=='Rotterdam')&(funda['BUURT']=='Groot-IJsselmonde')]='Groot IJsselmonde'
funda['BUURT'][(funda['GEMEENTE']=='Ridderkerk')&(funda['BUURT']=='Ridderkerk-Centrum')]='Centrum'
funda['BUURT'][(funda['GEMEENTE']=='Ridderkerk')&(funda['BUURT']=='Ridderkerk-Oost')]='Oost'
funda['BUURT'][(funda['GEMEENTE']=='Ridderkerk')&(funda['BUURT']=='Ridderkerk-West')]='West'
funda['BUURT'][(funda['GEMEENTE']=='Oisterwijk')&(funda['BUURT']=='Pannenschuur')]='Omg. Pannenschuur-Centrum'
funda['GEMEENTE'][funda['GEMEENTE']=='Nootdorp']='Pijnacker-Nootdorp'

In [20]:
# Delete rows from funda dataframe with undefined buurt and postcode
funda.dropna(subset=[['BUURT','POSTCODE_NUMBERS']],inplace=True)

In [21]:
funda.head()

,FILE_NAME,PROPERTY_TYPE,PROPERTY_ID,POSTCODE,GEMEENTE,BUURT,OFFER_DATE,SOLD_DATE,SOLD_PRICE,ASKING_PRICE,...,GARDEN_AREA,PARKING_TYPE,ENERGY_LABEL,MAKLER_NAME,MAKLER_TEL,HOUSE_LETTER,HOUSE_NUMBER,STREET,GARDEN_FLAG,POSTCODE_NUMBERS
0,appartement-4003351-boerhaavelaan-102-a-featur...,appartement,4003351,3112LM,Schiedam,Rotterdamsedijk,2008-01-15,2015-08-08,129000.0,149000.0,...,50.0,Openbaar parkeren en betaald parkeren,C,Makelaardij Wassenaar,010-4737933,A,102.0,BOERHAAVELAAN,1.0,3112
1,appartement-4043458-a-weg-10-25-features.html,appartement,4043458,9581AK,Musselkanaal,Zuid,2007-11-06,2016-03-14,247500.0,286500.0,...,NaN,NaN,B,Kral Westen Makelaardij B.V.,0599-581759,NaN,10.0,A WEG,0.0,9581
2,appartement-4047711-dingstraat-20-features.html,appartement,4047711,7101GX,Winterswijk,Centrale deel,2007-10-31,2015-11-11,180000.0,265000.0,...,NaN,NaN,B,Enning Makelaardij b.v.,0543-516151,NaN,20.0,DINGSTRAAT,0.0,7101
3,appartement-4096883-groningerstraat-72-d-featu...,appartement,4096883,9401BM,Assen,Galgenveld,2007-12-19,2016-01-04,175000.0,267550.0,...,NaN,NaN,A,Oosting Makelaars,0592-310144,D,72.0,GRONINGERSTRAAT,0.0,9401
4,appartement-4104014-weverstraat-11-features.html,appartement,4104014,7271AJ,Borculo,Borculo Oude-Centrum,2008-04-22,2015-09-21,125000.0,158500.0,...,NaN,NaN,C,Bijker makelaardij,0545-254444,NaN,11.0,WEVERSTRAAT,0.0,7271


In [22]:
cbsBuurt2015.head()

,GM_NAAM,BU_NAAM,POSTCODE,BU_CODE,WK_CODE,GM_CODE,WATER,AANTAL_HH,AANT_INW,AANT_MAN,...,P_VERWEDUW,P_WEST_AL,P_WONT2000,P_WONV2000,STED,STERFT_TOT,WONINGEN,WOZ,WWB_UITTOT,WW_UIT_TOT
0,'s-Gravenhage,Archipelbuurt,2585,BU05180546,WK051805,GM0518,NEE,3210.0,5720.0,2700.0,...,4.0,30.0,94.0,6.0,1.0,35.0,3591.0,376.0,60.0,90.0
1,'s-Gravenhage,Arendsdorp,2596,BU05180478,WK051804,GM0518,NEE,555.0,1310.0,625.0,...,9.0,29.0,100.0,0.0,1.0,35.0,866.0,483.0,10.0,20.0
2,'s-Gravenhage,Belgisch Park,2587,BU05180271,WK051802,GM0518,NEE,4130.0,7955.0,3935.0,...,5.0,23.0,99.0,1.0,2.0,110.0,4091.0,294.0,120.0,130.0
3,'s-Gravenhage,Bezuidenhout-Midden,2595,BU05182665,WK051826,GM0518,NEE,2120.0,3970.0,2000.0,...,3.0,27.0,80.0,20.0,1.0,20.0,2069.0,244.0,30.0,80.0
4,'s-Gravenhage,Bezuidenhout-Oost,2593,BU05182666,WK051826,GM0518,NEE,5175.0,8940.0,4315.0,...,3.0,22.0,98.0,2.0,1.0,45.0,5151.0,169.0,210.0,200.0


In [23]:
print cbsBuurt2015.shape
cbsBuurt2015.drop_duplicates(["BU_NAAM","POSTCODE"],inplace=True)
print cbsBuurt2015.shape

(12179, 127)
(12178, 127)


In [24]:
df = funda.merge(cbsBuurt2015, left_on=["BUURT","POSTCODE_NUMBERS"], right_on=["BU_NAAM","POSTCODE"], how="left")
print "DataFrame shape before merging is {}".format(funda.shape)
print "DataFrame shape after merging is {}".format(df.shape)


DataFrame shape before merging is (194555, 31)
DataFrame shape after merging is (194555, 158)


In [25]:
df.head()

,FILE_NAME,PROPERTY_TYPE,PROPERTY_ID,POSTCODE_x,GEMEENTE,BUURT,OFFER_DATE,SOLD_DATE,SOLD_PRICE,ASKING_PRICE,...,P_VERWEDUW,P_WEST_AL,P_WONT2000,P_WONV2000,STED,STERFT_TOT,WONINGEN,WOZ,WWB_UITTOT,WW_UIT_TOT
0,appartement-4003351-boerhaavelaan-102-a-featur...,appartement,4003351,3112LM,Schiedam,Rotterdamsedijk,2008-01-15,2015-08-08,129000.0,149000.0,...,3.0,22.0,100.0,0.0,1.0,15.0,1895.0,105.0,150.0,160.0
1,appartement-4043458-a-weg-10-25-features.html,appartement,4043458,9581AK,Musselkanaal,Zuid,2007-11-06,2016-03-14,247500.0,286500.0,...,12.0,3.0,90.0,10.0,4.0,35.0,655.0,153.0,10.0,40.0
2,appartement-4047711-dingstraat-20-features.html,appartement,4047711,7101GX,Winterswijk,Centrale deel,2007-10-31,2015-11-11,180000.0,265000.0,...,8.0,10.0,80.0,20.0,2.0,45.0,2108.0,140.0,140.0,120.0
3,appartement-4096883-groningerstraat-72-d-featu...,appartement,4096883,9401BM,Assen,Galgenveld,2007-12-19,2016-01-04,175000.0,267550.0,...,10.0,11.0,49.0,51.0,1.0,5.0,485.0,145.0,40.0,20.0
4,appartement-4104014-weverstraat-11-features.html,appartement,4104014,7271AJ,Borculo,Borculo Oude-Centrum,2008-04-22,2015-09-21,125000.0,158500.0,...,5.0,4.0,87.0,13.0,3.0,0.0,287.0,149.0,10.0,20.0


In [26]:
print df['PROPERTY_ID'][df['GM_NAAM'].isnull()].count()
df[df['GM_NAAM'].isnull()].head()


16125


,FILE_NAME,PROPERTY_TYPE,PROPERTY_ID,POSTCODE_x,GEMEENTE,BUURT,OFFER_DATE,SOLD_DATE,SOLD_PRICE,ASKING_PRICE,...,P_VERWEDUW,P_WEST_AL,P_WONT2000,P_WONV2000,STED,STERFT_TOT,WONINGEN,WOZ,WWB_UITTOT,WW_UIT_TOT
10,appartement-4165749-charloisse-hoofd-81-b-feat...,appartement,4165749,3087CA,Rotterdam,Oud-Charlois,2008-03-05,2016-05-02,197500.0,224000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,appartement-4231815-boulevard-1945-284-feature...,appartement,4231815,7511AH,Enschede,Getfert,2008-08-28,2016-02-15,115000.0,124000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,appartement-4252111-dr-joh-g-mezgerstraat-41-f...,appartement,4252111,2041HT,Zandvoort,Boulevard,2008-06-04,2016-06-10,192500.0,239000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,appartement-4278666-hogeweg-92-features.html,appartement,4278666,2042GJ,Zandvoort,Hogeweg-Haarlemmerstraat,2008-06-18,2016-08-04,995000.0,1295000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,appartement-4314873-uitweg-15-a-i-features.html,appartement,4314873,3051JP,Rotterdam,Hillegersberg-Zuid,2008-10-14,2016-04-11,135000.0,139500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
# Combine Funda and ZIPCODE database using POSTCODE as a key parameter

In [27]:
def float2int(x):
    if pd.isnull(x)==False:
        try:
            return int(x)
        except:
            return 0
    else:
        return 0
    
df['HOUSE_NUMBER'] = df['HOUSE_NUMBER'].apply(float2int)

In [28]:

df[['POSTCODE_x','POSTCODE_NUMBERS','STREET','HOUSE_NUMBER','HOUSE_LETTER']].head()

,POSTCODE_x,POSTCODE_NUMBERS,STREET,HOUSE_NUMBER,HOUSE_LETTER
0,3112LM,3112,BOERHAAVELAAN,102,A
1,9581AK,9581,A WEG,10,NaN
2,7101GX,7101,DINGSTRAAT,20,NaN
3,9401BM,9401,GRONINGERSTRAAT,72,D
4,7271AJ,7271,WEVERSTRAAT,11,NaN


In [29]:
zipcodes['OPENBARERUIMTE'] = zipcodes['OPENBARERUIMTE'].str.upper()
zipcodes['HOUSE_LETTER'] = zipcodes['HOUSE_LETTER'].str.upper()
zipcodes.head()

,OPENBARERUIMTE,HOUSE_NUMBER,HOUSE_LETTER,HOUSE_SUFFIX,POSTCODE,WOONPLATS,GEMEENTE,PROVINCE,OBJECT_ID,OBJECT_TYPE,SECONDARY_ADDRESS,X,Y,LON,LAT
0,DE RUIJTERKADE,105,NaN,1,1011AB,Amsterdam,Amsterdam,Noord-Holland,363010000964973,VBO,f,122177.0,487877.0,4.905167,52.377780
1,DE RUIJTERKADE,105,NaN,2,1011AB,Amsterdam,Amsterdam,Noord-Holland,363010000784610,VBO,f,122177.0,487877.0,4.905167,52.377780
2,DE RUIJTERKADE,105,NaN,3,1011AB,Amsterdam,Amsterdam,Noord-Holland,363010000977861,VBO,f,122177.0,487877.0,4.905167,52.377780
3,DE RUIJTERKADE,105,NaN,H,1011AB,Amsterdam,Amsterdam,Noord-Holland,363010000785105,VBO,f,122177.0,487877.0,4.905167,52.377780
4,DE RUIJTERKADE,106,NaN,1,1011AB,Amsterdam,Amsterdam,Noord-Holland,363010012064067,VBO,f,122185.0,487882.9,4.905284,52.377833


In [30]:
zipcodes.drop_duplicates(["POSTCODE","OPENBARERUIMTE","HOUSE_NUMBER","HOUSE_LETTER"],inplace=True)

In [31]:
print df.shape
df = df.merge(zipcodes[["POSTCODE","OPENBARERUIMTE","HOUSE_NUMBER","HOUSE_LETTER",'X','Y','LON','LAT']], left_on=["POSTCODE_x","STREET","HOUSE_NUMBER","HOUSE_LETTER"], right_on=["POSTCODE","OPENBARERUIMTE","HOUSE_NUMBER","HOUSE_LETTER"], how="left")
print df.shape

(194555, 158)
(194555, 164)


Calculate the missed average coordinates based on POSTCODE, STREET(OPENBARERUIMTE) and HOUSE_NUMBER attributes.
Make an attempt to use all available information. E.g. if the house number is unknow try to use only POSTCODE and STREET data etc.

In [32]:
def assign_average_geocoordinates(x):
    if pd.isnull(x['X']):
        if zipcodes[(zipcodes['POSTCODE']==x['POSTCODE_x'])&\
                    (zipcodes['OPENBARERUIMTE']==x['STREET'])&\
                    (zipcodes['HOUSE_NUMBER']==x['HOUSE_NUMBER'])].shape[0]>0:
            average = zipcodes[(zipcodes['POSTCODE']==x['POSTCODE_x'])&(zipcodes['OPENBARERUIMTE']==x['STREET'])&(zipcodes['HOUSE_NUMBER']==x['HOUSE_NUMBER'])][['X','Y','LON','LAT']].mean()
            x['X'] = average['X']
            x['Y'] = average['Y']
            x['LON'] = average['LON']
            x['LAT'] = average['LAT']
            return pd.Series({'POSTCODE_x':x['POSTCODE_x'],'STREET':x['STREET'],'HOUSE_NUMBER':x['HOUSE_NUMBER'],'X':x['X'],'Y':x['Y'],'LON':x['LON'],'LAT':x['LAT']})
        elif zipcodes[(zipcodes['POSTCODE']==x['POSTCODE_x'])&\
                    (zipcodes['OPENBARERUIMTE']==x['STREET'])].shape[0]>0:
            average = zipcodes[(zipcodes['POSTCODE']==x['POSTCODE_x'])&(zipcodes['OPENBARERUIMTE']==x['STREET'])][['X','Y','LON','LAT']].mean()
            x['X'] = average['X']
            x['Y'] = average['Y']
            x['LON'] = average['LON']
            x['LAT'] = average['LAT']
            return pd.Series({'POSTCODE_x':x['POSTCODE_x'],'STREET':x['STREET'],'HOUSE_NUMBER':x['HOUSE_NUMBER'],'X':x['X'],'Y':x['Y'],'LON':x['LON'],'LAT':x['LAT']})
        elif zipcodes[(zipcodes['POSTCODE']==x['POSTCODE_x'])].shape[0]>0:
            average = zipcodes[(zipcodes['POSTCODE']==x['POSTCODE_x'])][['X','Y','LON','LAT']].mean()
            x['X'] = average['X']
            x['Y'] = average['Y']
            x['LON'] = average['LON']
            x['LAT'] = average['LAT']
            return pd.Series({'POSTCODE_x':x['POSTCODE_x'],'STREET':x['STREET'],'HOUSE_NUMBER':x['HOUSE_NUMBER'],'X':x['X'],'Y':x['Y'],'LON':x['LON'],'LAT':x['LAT']})
        else:
            return pd.Series({'POSTCODE_x':x['POSTCODE_x'],'STREET':x['STREET'],'HOUSE_NUMBER':x['HOUSE_NUMBER'],'X':x['X'],'Y':x['Y'],'LON':x['LON'],'LAT':x['LAT']})
    else:
        return pd.Series({'POSTCODE_x':x['POSTCODE_x'],'STREET':x['STREET'],'HOUSE_NUMBER':x['HOUSE_NUMBER'],'X':x['X'],'Y':x['Y'],'LON':x['LON'],'LAT':x['LAT']})

In [33]:
df_with_na = df[df['X'].isnull()]
print df_with_na.shape

(31262, 164)


In [ ]:
df_with_na[['POSTCODE_x','STREET','HOUSE_NUMBER','X','Y','LON','LAT']] = df_with_na[['POSTCODE_x','STREET','HOUSE_NUMBER','X','Y','LON','LAT']].apply(assign_average_geocoordinates, axis=1)

In [90]:
df = pd.concat((df[df['X'].isnull()==False],df_with_na))

In [91]:
# Save DataFrame
df.to_pickle("Processed_data/funda_and_cbs.pickle")